In [1]:
import requests
from pymongo import MongoClient
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display

import urllib2
import sys

In [2]:
url = "https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police"

In [3]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [4]:
comments = soup.findAll("div", { "class" : "d-comment__body" })
len(comments)

0

In [7]:
driver = webdriver.PhantomJS('/media/sanhita/Windows7_OS/workspace/downloads/phantomjs-2.1.1-linux-x86_64/bin/phantomjs')
driver.get(url) #navigate to the page using the code
time.sleep(5)
html = driver.page_source  
driver.quit()
soup = BeautifulSoup(html, 'html.parser')

In [8]:
comments = soup.findAll("div", { "class" : "d-comment__body" })
len(comments)

34

In [7]:
# url = "https://www.theguardian.com/us/commentisfree"

# Which links to look for.

# years = ["2015", "2016", "2017"]
# months = ["jan", "feb"]
# dates = [str(n) for n in xrange(14,20)]

years = ["2017"]
months = ["oct"]
dates = [str(13)]


# count of number of links
i = 0

# Initiating mongoDB. Using pymongo to connect the database
client = MongoClient()
db = client["guardian"] #This is the name of the database
urls = db["urls"] # this is the table in that database

result = db.urls.delete_many({}) # A fresh start to the DB table -> removing all entries 
all_links = []

for month in months:
    for date in dates:
        root_url = "https://www.theguardian.com/commentisfree/2016/" + month + "/" + date 
        
        # This one contains all the URLs. Soup extracts those.
        list_of_urls = requests.get(root_url).text
        soup = BeautifulSoup(list_of_urls, "html.parser")

        # the .findAll method from re finds the _content_ that has 
        # html tag - 'a', attribute - 'href' and 
        # the pattern that begins with the root_url variable
        for link in soup.findAll('a', attrs={'href': re.compile(root_url)}):
            all_links.append(link.get('href'))
            
all_links = set(all_links)
for link in all_links:
    # urls is the name of the mongo DataBase. insert_one is the method 
    urls.insert_one({"id" : i, "url" : link})
#             print link
    i += 1
print "Total number of links stored is ", i

Total number of links stored is  21


In [3]:

i = 0
for d in urls.find():
    i += 1
print "Total number of links in the database table called url is", i


Total number of links in the database table called url is 29


In [4]:
def download_comments(guardianLink):
    '''
    This function downloads the comments, along with title, topic of the original thread
    comment_id, no_upvotes, no_replies, comment_author, comment_author_id, etc
    Returns BeautifulSoup; need more extraction from the soup for the above-mentioned 
    parameters
    
    '''
        
    #Figuring out the PageId
    pageResponse = urllib2.urlopen(guardianLink)
    commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
    # print "Comments Match = ", commentsMatch()
    if commentsMatch:
        pageId = commentsMatch.group(1)
        print '[+] pageId has been retrieved ('+pageId+')'
    else:
        sys.exit('[-] Could not retrieve pageId!')

    #Retrieving comments
    downloadCount = 1
    downloadError = 0

    startTime = time.time()
    text = ''

    print '[~] Start downloading pages ...'
    while downloadError == 0:
        try:
            response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId + '?page=' + str(downloadCount))
            html = response.read()
            length = len(text)
            text += html
#             if len(text) <= length:
#                 print "Text appending isn't working as expected"
#             else :
#                 print "Text appending may have worked"
            if downloadCount == 1:
                soup = BeautifulSoup(html, 'html.parser')
#             if downloadCount %5 == 0:
#                 print '[+] Page: ' + str(downloadCount) + ' has been downloaded!'
            downloadCount = downloadCount + 1

        except:
            downloadError = 1

    stopTime = time.time()
    runningTime = (stopTime - startTime)/60;
    print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes' 		
    return BeautifulSoup(text, 'html.parser')

In [19]:
def download_comments_selenium(guardianLink):
    '''
    This function downloads the comments, along with title, topic of the original thread
    comment_id, no_upvotes, no_replies, comment_author, comment_author_id, etc
    Returns BeautifulSoup; need more extraction from the soup for the above-mentioned 
    parameters
    
    '''
        
    #Figuring out the PageId
    pageResponse = urllib2.urlopen(guardianLink)
    commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
    # print "Comments Match = ", commentsMatch()
    if commentsMatch:
        pageId = commentsMatch.group(1)
        print '[+] pageId has been retrieved ('+pageId+')'
    else:
        sys.exit('[-] Could not retrieve pageId!')

    #Retrieving comments
    downloadCount = 1
    downloadError = 0

    startTime = time.time()
    text = ''

#     time.sleep(5) # it takes a while to load the entire page. So wait.

    
    print '[~] Start downloading pages ...'
    while downloadError == 0:
        try:
            url = 'http://www.theguardian.com/discussion/p/' + pageId + '?page=' + str(downloadCount)
            driver = webdriver.PhantomJS('/media/sanhita/Windows7_OS/workspace/downloads/phantomjs-2.1.1-linux-x86_64/bin/phantomjs')
            driver.get(url) #navigate to the page using the code
            
            length = len(text)
            text += driver.page_source  
            
            driver.quit()
            
            if len(text) <= length:
                print "Text appending isn't working as expected"
            else :
                print "Text appending may have worked"

            if downloadCount %5 == 0:
                print '[+] Page: ' + str(downloadCount) + ' has been downloaded!'

            downloadCount = downloadCount + 1
            
        except:
            downloadError = 1

    stopTime = time.time()
    runningTime = (stopTime - startTime)/60;
    
    print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes' 		
    return BeautifulSoup(text, 'html.parser')

In [109]:
def num_shares(url):
    '''
    This function reads the url using the phanton browser, phantom because it won't open a window,
    read the webpage and find the number of shares of the link.
    It takes a while to read this number; so this function is very slow. Takes at least 5 mins per URL.
    returns interger number of shares on the social media. 
    Put it in mongodb
    '''
    
    
    driver = webdriver.PhantomJS('/media/sanhita/Windows7_OS/workspace/downloads/phantomjs-2.1.1-linux-x86_64/bin/phantomjs')
    driver.get(url) #navigate to the page using the code; it takes a while to run this call
    time.sleep(5)
    html_source = driver.page_source 
    driver.quit()
    
    soup = BeautifulSoup(html_source, 'html.parser')
    for number in soup.find('div', {"class" : "sharecount__value--short"}):
        n = number
    if not number:            
        number = 0
    
    return int(n)

In [5]:
len(urls.distinct(key='url'))

29

In [ ]:
# To retrieve data from mongodb, using id as the identifier

for url in urls.find({'id': 0}):
    print url['url']

In [ ]:
urls.find({'id': 0})

In [4]:
num_shares('https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police')

NameError: name 'num_shares' is not defined

In [99]:
url = "https://www.theguardian.com/commentisfree/2017/aug/13/america-carpet-bombed-north-korea-remember-that-past"

text = requests.get(url).text
soup = BeautifulSoup(text, "html.parser")
# print soup.prettify()

In [82]:
soup.text + u'\ttest'

u'\n\n\n\nAmericans once carpet-bombed North Korea. It\'s time to remember that past | Bruce Cumings | Opinion | The Guardian\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        {"name":"The Guardian","url":"http://www.theguardian.com/","logo":"https://assets.guim.co.uk/images/favicons/451963ac2e23633472bf48e2856d3f04/152x152.png","sameAs":["https://www.facebook.com/theguardian","https://twitter.com/guardian","https://plus.google.com/+theGuardian","https://www.youtube.com/user/TheGuardian"],"@type":"Organization","@context":"http://schema.org"}\n    \n\n        {"@id":"https://www.theguardian.com/commentisfree/2017/aug/13/america-carpet-bombed-north-korea-remember-that-past","potentialAction":{"@type":"ViewAction","target":"android-app://com.guardian/https/www.theguardian.com/commentisfree/2017/aug/13/america-carpet-bombed-north-korea-remember-that-past"},"@type":"WebPage","@context":"htt

In [ ]:
cursor = urls.find({})
for document in cursor: 
    soup = download_comments(str(document['url']))
    print "Working on this link", document['url']
    auth_lst = []
    comm_id_lst = []
    auth_id_lst = []
    for lis in soup.find_all('li'):
        if 'data-comment-author-id' in (lis.attrs) and 'data-comment-id' in (lis.attrs)and 'data-comment-author' in (lis.attrs):
            auth_lst.append(lis.attrs['data-comment-author'].encode('utf-8').strip())
            auth_id_lst.append(int(lis.attrs['data-comment-author-id']))
            comm_id_lst.append(int(lis.attrs['data-comment-id']))



    print  "Numbers of Authors, Author ids, comment ids", len(auth_lst), len(auth_id_lst), len(comm_id_lst)

    comments = soup.findAll("div", { "class" : "d-comment__body" })
    recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
    users = soup.findAll("span", {"itemprop" : "givenName"})

    i = 0
    j = 0
    for com, rec, user, auth, auth_id, comm_id in zip(comments, recommends, users, auth_lst, auth_id_lst, comm_id_lst):
        i += 1
        if 'comment was removed by a moderator ' not in com.text:
            j += 1
                    

            if auth != user.text.encode('utf-8').strip():
                print "something is broken"
                

    #         break
    #     elif str(user.text) != auth:
    #         print com.text, rec.text, user.text, auth, auth_id, comm_id

    #     else : 
    #     print user.text
    print "The number of recommendations, users, comments is ", i
    print "The number of removed comments is ", i - j

[+] pageId has been retrieved (57ap9)
[~] Start downloading pages ...
[=] 5 pages have been downloaded in 0.09 minutes
Working on this link https://www.theguardian.com/commentisfree/2016/oct/13/snp-old-politics-ideaspace-independence-vote
Numbers of Authors, Author ids, comment ids 220 220 220
The number of recommendations, users, comments is  219
The number of removed comments is  1
[+] pageId has been retrieved (5772k)
[~] Start downloading pages ...
[=] 15 pages have been downloaded in 0.21 minutes
Working on this link https://www.theguardian.com/commentisfree/2016/oct/13/hinkley-hs2-heathrow-theresa-may-team-out-of-depth
Numbers of Authors, Author ids, comment ids 741 741 741
The number of recommendations, users, comments is  741
The number of removed comments is  0
[+] pageId has been retrieved (57byb)
[~] Start downloading pages ...
[=] 0 pages have been downloaded in 0.0 minutes
Working on this link https://www.theguardian.com/commentisfree/2016/oct/13/donald-trump-assault-alleg

In [6]:
url = 'https://www.theguardian.com/commentisfree/2016/oct/31/and-now-a-new-cruelty-is-heaped-upon-the-victims-of-our-nations-political-expedience' 

soup = download_comments(url)
print "Working on this link", url
auth_lst = []
comm_id_lst = []
auth_id_lst = []
for lis in soup.find_all('li'):
    if 'data-comment-author-id' in (lis.attrs) and 'data-comment-id' in (lis.attrs)and 'data-comment-author' in (lis.attrs):
        auth_lst.append(lis.attrs['data-comment-author'].encode('utf-8').strip())
        auth_id_lst.append(int(lis.attrs['data-comment-author-id']))
        comm_id_lst.append(int(lis.attrs['data-comment-id']))



print  "Numbers of Authors, Author ids, comment ids", len(auth_lst), len(auth_id_lst), len(comm_id_lst)

comments = soup.findAll("div", { "class" : "d-comment__body" })
recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = soup.findAll("span", {"itemprop" : "givenName"})

i = 0
j = 0
for com, rec, user, auth, auth_id, comm_id in zip(comments, recommends, users, auth_lst, auth_id_lst, comm_id_lst):
    i += 1
    if 'comment was removed by a moderator ' not in com.text:
        j += 1


        if auth != user.text.encode('utf-8').strip():
            print auth, user.text.encode('utf-8').strip()


#         break
#     elif str(user.text) != auth:
#         print com.text, rec.text, user.text, auth, auth_id, comm_id

#     else : 
#     print user.text
print "The number of recommendations, users, comments is ", i
print "The number of removed comments is ", i - j

[+] pageId has been retrieved (5a4pa)
[~] Start downloading pages ...
[=] 14 pages have been downloaded in 0.15 minutes
Working on this link https://www.theguardian.com/commentisfree/2016/oct/31/and-now-a-new-cruelty-is-heaped-upon-the-victims-of-our-nations-political-expedience
Numbers of Authors, Author ids, comment ids 673 673 673
Louise  Leotta Louise Leotta
The number of recommendations, users, comments is  660
The number of removed comments is  13


In [63]:
type(soup)

str

In [64]:
# url = "https://www.theguardian.com/commentisfree/2017/oct/23/houston-harvey-construction-workers-protect-abuse-undocumented"
url = 'https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police'
soup = download_comments(url)
print type(soup)



[+] pageId has been retrieved (4fz8p)
[~] Start downloading pages ...
Text appending may have worked
Text appending may have worked
Text appending may have worked
Text appending may have worked
Text appending may have worked
[+] Page: 5 has been downloaded!
Text appending may have worked
Text appending may have worked
Text appending may have worked
[=] 8 pages have been downloaded in 0.08 minutes
<class 'bs4.BeautifulSoup'>


In [55]:

auth_lst = []
comm_id_lst = []
auth_id_lst = []
for lis in soup.find_all('li'):
    if 'data-comment-author-id' in (lis.attrs) and 'data-comment-id' in (lis.attrs)and 'data-comment-author' in (lis.attrs):
        auth_lst.append(lis.attrs['data-comment-author'].encode('utf-8').strip())
        auth_id_lst.append(int(lis.attrs['data-comment-author-id']))
        comm_id_lst.append(int(lis.attrs['data-comment-id']))
        


print  "Numbers of Authors, Author ids, comment ids", len(auth_lst), len(auth_id_lst), len(comm_id_lst)

comments = soup.findAll("div", { "class" : "d-comment__body" })
recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = soup.findAll("span", {"itemprop" : "givenName"})

i = 0
j = 0
for com, rec, user, auth, auth_id, comm_id in zip(comments, recommends, users, auth_lst, auth_id_lst, comm_id_lst):
    i += 1
    if 'comment was removed by a moderator ' not in com.text:
        j += 1
        if auth != user.text.encode('utf-8').strip():
            print auth, user.text.encode('utf-8').strip()
        
#         break
#     elif str(user.text) != auth:
#         print com.text, rec.text, user.text, auth, auth_id, comm_id
        
#     else : 
#     print user.text
print "The number of recommendations, users, comments is ", i
print "The number of removed comments is ", i - j

Numbers of Authors, Author ids, comment ids 380 380 380
The number of recommendations, users, comments is  368
The number of removed comments is  12


In [50]:
auth[0]

u'C'

In [30]:
for com, rec, user in zip(comments, recommends, users):
    if 'comment was removed by a moderator ' in com.text:
        print com.text, recommend.text



This comment was removed by a moderator because it didn't abide by our
community standards.
Replies may also be deleted. For more detail see our FAQs.
8


This comment was removed by a moderator because it didn't abide by our
community standards.
Replies may also be deleted. For more detail see our FAQs.
8


This comment was removed by a moderator because it didn't abide by our
community standards.
Replies may also be deleted. For more detail see our FAQs.
8


This comment was removed by a moderator because it didn't abide by our
community standards.
Replies may also be deleted. For more detail see our FAQs.
8


This comment was removed by a moderator because it didn't abide by our
community standards.
Replies may also be deleted. For more detail see our FAQs.
8


This comment was removed by a moderator because it didn't abide by our
community standards.
Replies may also be deleted. For more detail see our FAQs.
8


This comment was removed by a moderator because it didn't abide by o

In [114]:
url = "https://www.theguardian.com/commentisfree/2017/oct/23/houston-harvey-construction-workers-protect-abuse-undocumented"
# url = 'https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police'

soup2 = download_comments_selenium(url)

[+] pageId has been retrieved (7e69n)
[~] Start downloading pages ...
Text appending may have worked
[=] 1 pages have been downloaded in 0.12 minutes


In [115]:
soup2.prettify()

u'<!DOCTYPE html>\n<html class="js-on is-modern id--signed-out svg has-flex has-flex-wrap has-fixed has-sticky should-kern" data-page-path="/discussion/p/7e69n" id="js-context" lang="en">\n <head>\n  <title>\n   Who will protect the workers cleaning up Houston? | Discussion | The Guardian\n  </title>\n  <meta charset="utf-8">\n   <meta content="IE=Edge" http-equiv="X-UA-Compatible">\n    <meta content="telephone=no" name="format-detection">\n     <meta content="True" name="HandheldFriendly">\n      <meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport">\n       <link href="https://assets.guim.co.uk/" rel="dns-prefetch">\n        <link href="https://i.guim.co.uk" rel="dns-prefetch">\n         <link href="https://api.nextgen.guardianapps.co.uk" rel="dns-prefetch">\n          <link href="https://hits-secure.theguardian.com" rel="dns-prefetch">\n           <link href="//j.ophan.co.uk" rel="dns-prefetch">\n            <link href="//ophan.theguardian.com" rel="dns

In [16]:
comments = soup.findAll("div", { "class" : "d-comment__body" })
recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = soup.findAll("span", {"itemprop" : "givenName"})
replies = soup.findAll("span", {"class" : "d-comment__reply-to-author"})
links = soup.findAll("a", {"itemprop" : "d-comment__reply-to-author"})

i = 0
for user, comment, recommend, reply in zip(users, comments, recommends, replies):
    if i == 100:
        print user.text, comment.text, int(recommend.text), reply.text
    i += 1
print i

Erik Frederiksen 
Tides make boats go up and down.
We’re already committed to around 6m of sea level rise.
One foot of sea level rise, averaged globally, moves the shoreline 300 feet inland and caused Hurricane Sandy to flood an additional 25 square miles.
6 Erik Frederiksen
106


In [117]:
comments = soup2.findAll("div", { "class" : "d-comment__body" })
recommends = soup2.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = soup2.findAll("span", {"itemprop" : "givenName"})
replies = soup2.findAll("span", {"class" : "d-comment__reply-to-author"})
links = soup2.findAll("a", {"itemprop" : "d-comment__reply-to-author"})

i = 0
for user, comment, recommend, reply in zip(users, comments, recommends, replies):
    if i == 10:
        print user.text, comment.text, int(recommend.text), reply.text
    i += 1
print i

EdChamp 

I don't see why Mexican workers, who are mostly indigenous, have less right to live and work in what is now the Southwest of the United Snakes than have the heirs and descendants of the Anglophone settler-colonists who took the land by violence from indigenous peoples

That's easy enough to answser: they aren't American citizens, and they have to obey the laws of the US on immigration, just like every other nation in the world. Or, do you think they are somehow "special" and don't need to obey the law? It's almost a joke that some people refer to the belief that the vast majority of illegals (sorry, I meant illegal aliens) are law-abiding.
2 Aaron Aarons
90


In [118]:
i = 0
lst = []
for lis in soup2.find_all('li'):
    if 'data-comment-author-id' in (lis.attrs) and 'data-comment-id' in (lis.attrs)and 'data-comment-author' in (lis.attrs):
        print lis.attrs['data-comment-id'], lis.attrs['data-comment-author-id'], lis.attrs['data-comment-author'], lis.attrs['data-comment-replies']
#         lst.append(lis.attrs['data-comment-author'])
#         print lis.attrs
        
        i += 1

print  i

107372761 14555058 Gerryofhamilton 0
107362897 16262897 Wolframite 0
107355223 17443130 LocknessMunster 0
107328563 12274172 colddeadhands33 0
107326769 10150584 MerelyGifted 0
107317608 16262897 Wolframite 0
107310409 17005674 Form400 0
107309859 17005674 Form400 0
107309657 17005674 Form400 0
107296751 17048054 SFromNl 0
107294717 13914349 EdChamp 0
107294559 2557946 Voltaire21 0
107292237 16262897 Wolframite 0
107292178 16262897 Wolframite 0
107292063 17757589 620please 0
107292007 17757589 620please 0
107291357 13914349 EdChamp 0
107288640 10527918 Aaron Aarons 0
107288255 10527918 Aaron Aarons 0
107288107 10527918 Aaron Aarons 0
107287818 10527918 Aaron Aarons 0
107286322 16262897 Wolframite 0
107285774 15026801 zootsuitbeatnick 0
107283953 17114862 JayThomas 0
107283947 17048054 SFromNl 0
107283924 17048054 SFromNl 0
107283825 17114862 JayThomas 0
107282908 16262897 Wolframite 0
107282009 12274172 colddeadhands33 0
107281946 17048054 SFromNl 0
107280254 14933002 ehmaybe 0
1072798

In [128]:
url = 'https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [131]:
soup.prettify()

u'<!DOCTYPE html>\n<html class="js-off is-not-modern id--signed-out" data-page-path="/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police" id="js-context" lang="en">\n <head>\n  <title>\n   Who can prevent British women joining Isis? Those who have returned | Melanie Smith | Opinion | The Guardian\n  </title>\n  <meta charset="utf-8">\n   <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n   <meta content="telephone=no" name="format-detection"/>\n   <meta content="True" name="HandheldFriendly"/>\n   <meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport">\n    <link href="https://assets.guim.co.uk/" rel="dns-prefetch"/>\n    <link href="https://i.guim.co.uk" rel="dns-prefetch"/>\n    <link href="https://api.nextgen.guardianapps.co.uk" rel="dns-prefetch"/>\n    <link href="https://hits-secure.theguardian.com" rel="dns-prefetch"/>\n    <link href="//j.ophan.co.uk" rel="dns-prefetch"/>\n    <link href="//ophan.theguardian.com" rel="dns-pre

In [57]:
len(comments)

127

In [44]:
user_lst = []
for user in users:
    user_lst.append(user.text)

In [119]:
i = 0
lst = []
for lis in soup.find_all('li'):
    if 'data-comment-author-id' in (lis.attrs) and 'data-comment-id' in (lis.attrs)and 'data-comment-author' in (lis.attrs):
        print lis.attrs['data-comment-id'], lis.attrs['data-comment-author-id'], lis.attrs['data-comment-author'], lis.attrs['data-comment-replies']
#         lst.append(lis.attrs['data-comment-author'])
#         print lis.attrs
        
        i += 1

print  i

107372761 14555058 Gerryofhamilton 0
107362897 16262897 Wolframite 0
107355223 17443130 LocknessMunster 0
107328563 12274172 colddeadhands33 0
107326769 10150584 MerelyGifted 0
107317608 16262897 Wolframite 0
107310409 17005674 Form400 0
107309859 17005674 Form400 0
107309657 17005674 Form400 0
107296751 17048054 SFromNl 0
107294717 13914349 EdChamp 0
107294559 2557946 Voltaire21 0
107292237 16262897 Wolframite 0
107292178 16262897 Wolframite 0
107292063 17757589 620please 0
107292007 17757589 620please 0
107291357 13914349 EdChamp 0
107288640 10527918 Aaron Aarons 0
107288255 10527918 Aaron Aarons 0
107288107 10527918 Aaron Aarons 0
107287818 10527918 Aaron Aarons 0
107286322 16262897 Wolframite 0
107285774 15026801 zootsuitbeatnick 0
107283953 17114862 JayThomas 0
107283947 17048054 SFromNl 0
107283924 17048054 SFromNl 0
107283825 17114862 JayThomas 0
107282908 16262897 Wolframite 0
107282009 12274172 colddeadhands33 0
107281946 17048054 SFromNl 0
107280254 14933002 ehmaybe 0
1072798

In [126]:
for lis in soup.find_all('li'):
#     if '107214961' in lis.attrs['data-comment-id']:
#         print lis
        print lis.attrs

{u'class': [u'dropdown-menu__item']}
{u'class': [u'dropdown-menu__item']}
{u'class': [u'dropdown-menu__item']}
{u'class': [u'dropdown-menu__item']}
{u'class': [u'dropdown-menu__item', u'u-h', u'js-show-comment-activity']}
{u'class': [u'dropdown-menu__item']}
{u'class': [u'dropdown-menu__item']}
{u'class': [u'dropdown-menu__item']}
{u'class': [u'dropdown-menu__item']}
{u'class': [u'pillars__item']}
{u'class': [u'pillars__item']}
{u'class': [u'pillars__item']}
{u'class': [u'pillars__item']}
{u'class': [u'pillars__item']}
{u'data-section-name': u'news', u'role': u'menuitem', u'class': [u'menu-item', u'js-navigation-item']}
{u'role': u'menuitem', u'class': [u'menu-item', u'menu-item--home']}
{u'role': u'menuitem', u'class': [u'menu-item']}
{u'role': u'menuitem', u'class': [u'menu-item']}
{u'role': u'menuitem', u'class': [u'menu-item']}
{u'role': u'menuitem', u'class': [u'menu-item']}
{u'role': u'menuitem', u'class': [u'menu-item']}
{u'role': u'menuitem', u'class': [u'menu-item']}
{u'role':

In [48]:
# Check if the two types of scraping gives the same solution
[(u1, u2) for u1, u2 in zip(lst, user_lst) if u1 != u2 ] 

[]

In [74]:
num_replies = soup.findAll("div", { "class" : "d-comment d-comment--has-replies d-comment--top-level" })
for num in num_replies:
    print num.text


In [69]:
for lis in soup.find_all('li'):
    if 'data-comment-replies' in (lis.attrs) and int(lis.attrs['data-comment-replies']) > 0:
        print lis.attrs['data-comment-replies']

In [75]:
data = soup.findAll('li', {'class':'d-comment d-comment--has-replies d-comment--top-level'})
print data

[]


In [ ]:

for i in xrange (len(lst)):
    if 'data-comment-author-id' in lst[i]:
        print i


In [ ]:
int(recommend.text)

In [ ]:
len(replies), len(users), len(comments), len(recommends)

In [ ]:
print soup.find('li', attrs={'id': 'data-comment-author-id'})

In [ ]:
f = open("GuardianDownload_4fmtq_1508968524.96_test.html", 'r')
text = f.read()
f.close()

In [ ]:
s2 = BeautifulSoup(text, 'html.parser')
comments = s2.findAll("div", { "class" : "d-comment__body" })
recommends = s2.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = s2.findAll("span", {"itemprop" : "givenName"})
replies = s2.findAll("span", {"class" : "d-comment__reply-to-author"})

len(replies), len(users), len(comments), len(recommends)

In [ ]:
url = "https://www.theguardian.com/us/commentisfree"

list_of_urls = requests.get(url).text

soup2 = BeautifulSoup(list_of_urls, "html.parser")


import re
i = 0
for link in soup2.findAll('a', attrs={'href': re.compile("https://www.theguardian.com/commentisfree/2017")}):
    print link.get('href')
    i += 1
print i

In [ ]:
url = 'https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police'

In [ ]:
soup = BeautifulSoup(urllib2.urlopen(url).read(), 'html.parser')

In [ ]:
i = 0
lst = []
for lis in soup.find_all('a'):
    if 'data-link-name' in (lis.attrs) and 'submeta__link' in lis.attrs:
        print lis.attrs['data-link-name']
        i += 1

print  i


# for topic in soup.find_all("a", { "class" : "data-link-name" }):
#     print topic

In [ ]:
for topic in soup.findAll("a", {"class" : "submeta__link"}):
    if 'data-link-name' in topic.attrs and 'keyword: ' in topic.attrs['data-link-name']:
        print topic.attrs['data-link-name'][9:]
# print section


In [ ]:
set(section)

In [8]:
for s in soup.find("h1", {"class" : "content__headline"}):
    print s

TypeError: 'NoneType' object is not iterable

# Do not run the cell below this one

In [ ]:
# The one below is a github code for downloading comments. Do not use this; 
# instead use a modified version of this in the cell below this one.

import urllib2
import re
import time
import sys

# This script will download all comments from a Guardian (theguardian.com) article into a single .html file
# Use 'File ID' to easily identify your file later (this could also be something like a Citavi identifier)
# Ingo Kleiber <ikleiber@googlemail.com>
# https://gist.github.com/IngoKl/3ebd968ef3c6da406638

print """\
   _____                     _ _             _____  _      
  / ____|                   | (_)           |  __ \| |     
 | |  __ _   _  __ _ _ __ __| |_  __ _ _ __ | |  | | |     
 | | |_ | | | |/ _` | '__/ _` | |/ _` | '_ \| |  | | |     
 | |__| | |_| | (_| | | | (_| | | (_| | | | | |__| | |____ 
  \_____|\__,_|\__,_|_|  \__,_|_|\__,_|_| |_|_____/|______|
"""                                                         

print 'Guardian Comment Downloader v.1.1 - Ingo Kleiber (05.01.2015)\n'
guardianLink = raw_input ('> Guadian Link: ')
fileId = raw_input('> File ID: ')

#Figuring out the PageId
pageResponse = urllib2.urlopen(guardianLink)
commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
# print "Comments Match = ", commentsMatch()
if commentsMatch:
    pageId = commentsMatch.group(1)
    print '[+] pageId has been retrieved ('+pageId+')'
else:
    sys.exit('[-] Could not retrieve pageId!')

#Retrieving comments
downloadCount = 1
downloadError = 0

startTime = time.time()
fileName = 'GuardianDownload_' + pageId + '_' + str(startTime) + '_' + fileId + '.html'
f = open(fileName, 'a')


print '[~] Start downloading pages ...'
while downloadError == 0:
    try:
        response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId + '?page=' + str(downloadCount))
        html = response.read()
        f.write(html)
        if downloadCount == 1:
            soup = BeautifulSoup(html, 'html.parser')
        print '[+] Page: ' + str(downloadCount) + ' has been downloaded!'
        downloadCount = downloadCount + 1
        
    except:
        downloadError = 1

stopTime = time.time()
runningTime = (stopTime - startTime)/60;
print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes' 		
f.close() 